In [1]:
from __future__ import print_function
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
if __name__ == "__main__":
    spark = SparkSession\
            .builder\
            .appName("LinearRegWithSpark")\
            .getOrCreate()

In [3]:
dataset = spark.read.csv("Admission_Prediction.csv",header=True)

In [4]:
dataset.show()

+---------+-----------+-----------------+----+----+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating| SOP| LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+----+----+----+--------+---------------+
|   337.00|     118.00|                4|4.50|4.50|9.65|    1.00|           0.92|
|   324.00|     107.00|                4|4.00|4.50|8.87|    1.00|           0.76|
|     null|     104.00|                3|3.00|3.50|8.00|    1.00|           0.72|
|   322.00|     110.00|                3|3.50|2.50|8.67|    1.00|           0.80|
|   314.00|     103.00|                2|2.00|3.00|8.21|    0.00|           0.65|
|   330.00|     115.00|                5|4.50|3.00|9.34|    1.00|           0.90|
|   321.00|     109.00|             null|3.00|4.00|8.20|    1.00|           0.75|
|   308.00|     101.00|                2|3.00|4.00|7.90|    0.00|           0.68|
|   302.00|     102.00|                1|2.00|1.50|8.00|    0.00|           0.50|
|   323.00|     

In [5]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [6]:
dataset.printSchema()

root
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR: string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit: string (nullable = true)



In [7]:
from pyspark.sql.functions import col
new_data = dataset.select(*(col(c).cast("float").alias(c) for c in dataset.columns))

In [8]:
new_data.printSchema()

root
 |-- GRE Score: float (nullable = true)
 |-- TOEFL Score: float (nullable = true)
 |-- University Rating: float (nullable = true)
 |-- SOP: float (nullable = true)
 |-- LOR: float (nullable = true)
 |-- CGPA: float (nullable = true)
 |-- Research: float (nullable = true)
 |-- Chance of Admit: float (nullable = true)



In [9]:
new_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|
|     null|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|
|    321.0|      109.0|             null|3.0|4.0| 8.2|     1.0|           0.75|
|    308.0|      101.0|              2.0|3.0|4.0| 7.9|     0.0|           0.68|
|    302.0|      102.0|              1.0|2.0|1.5| 8.0|     0.0|            0.5|
|    323.0|      108.0|              3.0

In [10]:
from pyspark.sql.functions import col,count,isnan,when

In [11]:
new_data.select([count(when(col(c).isNull(), c)).alias(c) for c in new_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|       15|         10|               15|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [12]:
from pyspark.ml.feature import Imputer

In [13]:
imputer = Imputer(inputCols=["GRE Score","TOEFL Score","University Rating"],
                 outputCols=["GRE Score","TOEFL Score","University Rating"])
model = imputer.fit(new_data)

imputed_data = model.transform(new_data)

In [14]:
#checking for null ir nan type values in our columns
imputed_data.select([count(when(col(c).isNull(), c)).alias(c) for c in imputed_data.columns]).show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|        0|          0|                0|  0|  0|   0|       0|              0|
+---------+-----------+-----------------+---+---+----+--------+---------------+



In [15]:
features = imputed_data.drop('Chance of Admit')

In [16]:
#lets assemble our features together using vector assembler
assembler = VectorAssembler(
            inputCols=features.columns,
            outputCol="features")

In [17]:
output = assembler.transform(imputed_data)

In [18]:
output.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|            features|
+---------+-----------+-----------------+---+---+----+--------+---------------+--------------------+
|    337.0|      118.0|              4.0|4.5|4.5|9.65|     1.0|           0.92|[337.0,118.0,4.0,...|
|    324.0|      107.0|              4.0|4.0|4.5|8.87|     1.0|           0.76|[324.0,107.0,4.0,...|
|316.55878|      104.0|              3.0|3.0|3.5| 8.0|     1.0|           0.72|[316.558776855468...|
|    322.0|      110.0|              3.0|3.5|2.5|8.67|     1.0|            0.8|[322.0,110.0,3.0,...|
|    314.0|      103.0|              2.0|2.0|3.0|8.21|     0.0|           0.65|[314.0,103.0,2.0,...|
|    330.0|      115.0|              5.0|4.5|3.0|9.34|     1.0|            0.9|[330.0,115.0,5.0,...|
|    321.0|      109.0|        3.1216495|3.0|4.0| 8.2|     1.0|           0.75|[321.0,109.0

In [19]:
output= output.select("features", "Chance of Admit")

In [20]:
train_df,test_df = output.randomSplit([0.7, 0.3])

In [21]:
train_df.show()
test_df.show()

+--------------------+---------------+
|            features|Chance of Admit|
+--------------------+---------------+
|[290.0,100.0,1.0,...|           0.47|
|[290.0,104.0,4.0,...|           0.45|
|[293.0,97.0,2.0,2...|           0.64|
|[294.0,93.0,1.0,1...|           0.46|
|[294.0,95.0,1.0,1...|           0.49|
|[295.0,99.0,1.0,2...|           0.37|
|[295.0,99.0,2.0,2...|           0.57|
|[295.0,101.0,2.0,...|           0.69|
|[296.0,95.0,2.0,3...|           0.44|
|[296.0,97.0,2.0,1...|           0.49|
|[296.0,99.0,2.0,2...|           0.61|
|[296.0,99.0,2.0,3...|           0.47|
|[296.0,101.0,1.0,...|            0.6|
|[297.0,96.0,2.0,2...|           0.43|
|[297.0,96.0,2.0,2...|           0.34|
|[297.0,98.0,2.0,2...|           0.59|
|[297.0,99.0,4.0,3...|           0.54|
|[297.0,100.0,1.0,...|           0.52|
|[297.0,101.0,3.0,...|           0.57|
|[298.0,92.0,1.0,2...|           0.51|
+--------------------+---------------+
only showing top 20 rows

+--------------------+---------------+

In [22]:
lin_reg = LinearRegression(featuresCol = 'features', labelCol='Chance of Admit')
linear_model = lin_reg.fit(train_df)

In [23]:
print("Coefficients: " + str(linear_model.coefficients))
print("Intercept: " + str(linear_model.intercept))

Coefficients: [0.0012247198428869356,0.0024620283745980668,0.007568789666640603,0.0009036849614507058,0.02131677944128885,0.12685539346014768,0.02596656201534931]
Intercept: -1.1333216723413488


In [24]:
trainSummary = linear_model.summary
print("RMSE: %f" % trainSummary.rootMeanSquaredError)
print("r2: %f" % trainSummary.r2)

RMSE: 0.058826
r2: 0.832270


In [25]:
# prediction
predictions = linear_model.transform(test_df)
predictions.select("prediction","Chance of Admit","features").show()

+-------------------+---------------+--------------------+
|         prediction|Chance of Admit|            features|
+-------------------+---------------+--------------------+
|0.42230784733735427|           0.46|[295.0,93.0,1.0,2...|
|0.45457067828382725|           0.47|[295.0,96.0,2.0,1...|
|0.47447380161509045|           0.45|[298.0,97.0,3.121...|
| 0.6225472932952458|           0.58|[298.0,100.0,3.0,...|
| 0.6102541302119606|           0.53|[298.0,101.0,4.0,...|
| 0.6846392596124216|           0.69|[298.0,105.0,3.0,...|
| 0.5210309833670315|           0.46|[298.0,107.187751...|
| 0.5254343598350772|           0.54|[299.0,96.0,2.0,1...|
| 0.5315193115965888|           0.59|[299.0,100.0,1.0,...|
| 0.5088035045409356|           0.42|[299.0,100.0,3.0,...|
| 0.6784197688901088|           0.56|[299.0,102.0,3.0,...|
| 0.6082053028362817|           0.65|[300.0,97.0,2.0,3...|
|  0.555711243491932|           0.58|[300.0,99.0,1.0,1...|
| 0.6290695884356547|           0.62|[300.0,100.0,3.12..

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator
pred_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="Chance of Admit",metricName="r2")
print("R Squared (R2) on test data = %g" % pred_evaluator.evaluate(predictions))

R Squared (R2) on test data = 0.778287


In [27]:
spark.stop()